# Splunk App for Data Science and Deep Learning - Notebook Template 

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with you Splunk platform by utilizing the Splunk App for Data Science and Deep Learning (DSDL) - formerly known as the Deep Learning Toolkit for Splunk (DLTK). Find more examples and information in the app and on the [DSDL splunkbase page](https://splunkbase.splunk.com/app/4607/#/details).

Note: By default every time you save this notebook the stages main cells are exported into a python module which can then get invoked by Splunk's MLTK SPL commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the DSDL app for more information about this workflow.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.21.6
pandas version: 1.3.5


## Stage 1 - get a data sample from Splunk
There are currently 2 ways to retrieve data from Splunk: Option 1 is to interactively pull data from Splunk into the DSDL Jupyter Lab environment. This is useful when the Splunk REST API is accessible from the Jupyter environment and a valid Splunk auth token is defined in the DSDL app. This option has advantages to quickly experiment with different Splunk SPL queries and further interactively work with the search results in Jupyter.

### Option 1 - pull data from Splunk

In [3]:
from dsdlsupport import SplunkSearch as SplunkSearch

In [4]:
search = SplunkSearch.SplunkSearch()

In [5]:
df = search.as_df()
df

,_time,feature_0,feature_1,feature_2,i,s
0,2022-10-05T16:35:55.000+02:00,0.6618700036779046,0.6140820817090571,0.4690734758041799,1,1
1,2022-10-05T16:35:55.000+02:00,0.8563643363304436,0.7017172779887915,0.7016464993357658,2,2
2,2022-10-05T16:35:55.000+02:00,0.18256376637145877,0.09785941755399108,0.7224264834076166,3,0
3,2022-10-05T16:35:55.000+02:00,0.7281667641364038,0.8856996349059045,0.40752001153305173,4,1
4,2022-10-05T16:35:55.000+02:00,0.8514458956196904,0.6438297093845904,0.9863259824924171,5,2
5,2022-10-05T16:35:55.000+02:00,0.9644769541919231,0.01729395193979144,0.1849918649531901,6,0
6,2022-10-05T16:35:55.000+02:00,0.4588743573985994,0.14953851513564587,0.6075516082346439,7,1
7,2022-10-05T16:35:55.000+02:00,0.13256517611443996,0.3875590469688177,0.7599751590751112,8,2
8,2022-10-05T16:35:55.000+02:00,0.3599444506689906,0.26945265382528305,0.42268069507554173,9,0
9,2022-10-05T16:35:55.000+02:00,0.9218343454413116,0.723359493073076,0.45556102972477674,10,1


### Option 2 - push data from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. You utilize the `mode=stage` flag in the in the `| fit` command to do this. The search results are accessible then as csv file with the same model name that is defined in the `into app:<modelname>` part of the fit statement. Additionally, meta data is retrieved and accessible as json file. In the same way you can further work with the meta data object as it is exposed in the fit and apply function definitions below in stage 3 and 4.

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [5]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [6]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("barebone_template")

In [7]:
df

,_time,i,feature_1,feature_2,feature_0
0,1665490693,1,0.252490,0.262905,0.641077
1,1665490693,2,0.291893,0.478008,0.070793
2,1665490693,3,0.578795,0.345771,0.702467
3,1665490693,4,0.892171,0.158889,0.813074
4,1665490693,5,0.299321,0.286945,0.342067
5,1665490693,6,0.851644,0.612204,0.477051
6,1665490693,7,0.229843,0.483297,0.858289
7,1665490693,8,0.607086,0.644280,0.819850
8,1665490693,9,0.723471,0.865462,0.870901
9,1665490693,10,0.713690,0.701663,0.117287


In [8]:
param

{'options': {'params': {'mode': 'stage', 'algo': 'barebone_template'},
  'args': ['_time', 'feature_*', 'i'],
  'feature_variables': ['_time', 'feature_*', 'i'],
  'model_name': 'barebone_template',
  'algo_name': 'MLTKContainer',
  'mlspl_limits': {'handle_new_cat': 'default',
   'max_distinct_cat_values': '100',
   'max_distinct_cat_values_for_classifiers': '100',
   'max_distinct_cat_values_for_scoring': '100',
   'max_fit_time': '600',
   'max_inputs': '100000',
   'max_memory_usage_mb': '4000',
   'max_model_size_mb': '30',
   'max_score_time': '600',
   'use_sampling': 'true'},
  'kfold_cv': None},
 'feature_variables': ['_time', 'feature_1', 'feature_2', 'feature_0', 'i']}

## Stage 2 - create and initialize a model

In [9]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['hyperparameter'] = 42.0
    return model

In [10]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{'hyperparameter': 42.0}


## Stage 3 - fit the model

In [11]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [12]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': 'model trained'}


## Stage 4 - apply the model

In [13]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    y_hat = df.index
    result = pd.DataFrame(y_hat, columns=['index'])
    return result

In [14]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

   index
0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9


## Stage 5 - save the model

In [15]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [16]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [17]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [18]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [19]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [20]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


In [21]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [22]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


## End of Stages
All subsequent cells are not tagged and can be used for further freeform code